# Exploring Airports in Australia

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import geoapify_key

## DataFrame Preparation

In [4]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("../Resources/cities.csv")
cities_pd.head()

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide


In [6]:
# Add columns the airports data we will fetch using the Geoapify API
# Note that we used "" to specify initial entry.
cities_pd["Lat"] = ""
cities_pd["Lon"] = ""
cities_pd["Airport Name"] = ""
cities_pd["IATA Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Distance"] = ""
cities_pd["Website"] = ""
cities_pd

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,
5,Gold Coast,,,,,,,
6,Townsville,,,,,,,
7,Williamtown,,,,,,,


## Look for Geographical Coordinates for Each City

In [16]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [18]:
# Print a message to follow up the airport search
print("Starting airport search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in cities_pd.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    city = row["City"] + ", Australia"
    # print(city)

    # Add the current city to the parameters
    params["text"] = city
    
    # Make the API request
    response = requests.get(base_url, params=params)
    print(response.url)
    
    # Convert response to JSON
    response = response.json()


    # Extract latitude and longitude
    cities_pd.loc[index, "Lat"] = response["results"][0]["lat"]
    cities_pd.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
cities_pd.head()

Starting airport search
https://api.geoapify.com/v1/geocode/search?apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&format=json&text=Sydney%2C+Australia
Coordinates for Sydney, Australia fetched...
https://api.geoapify.com/v1/geocode/search?apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&format=json&text=Melbourne%2C+Australia
Coordinates for Melbourne, Australia fetched...
https://api.geoapify.com/v1/geocode/search?apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&format=json&text=Perth%2C+Australia
Coordinates for Perth, Australia fetched...
https://api.geoapify.com/v1/geocode/search?apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&format=json&text=Cairns%2C+Australia
Coordinates for Cairns, Australia fetched...
https://api.geoapify.com/v1/geocode/search?apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&format=json&text=Adelaide%2C+Australia
Coordinates for Adelaide, Australia fetched...
https://api.geoapify.com/v1/geocode/search?apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&format=json&text=Gold+Coast%2C+Australia
Coordinates for Gol

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,,,,,
1,Melbourne,-37.814218,144.963161,,,,,
2,Perth,-31.955896,115.86058,,,,,
3,Cairns,-16.920666,145.772185,,,,,
4,Adelaide,-34.928181,138.599931,,,,,


## Retrieve Airports' Information

In [19]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for index, row in cities_pd.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lon"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
           
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    print(response.url)

        
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first airport from the results and store the details in the DataFrame
    try:
        cities_pd.loc[index, "Airport Name"] = response["features"][0]["properties"]["name"]
        cities_pd.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
        cities_pd.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        cities_pd.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
        cities_pd.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]

    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {cities_pd.loc[index, 'Airport Name']}")
        
    # Log the search results
    print(f"nearest airport from {city}: {cities_pd.loc[index, 'Airport Name']}")
    

# Display sample data


Starting airport details search
https://api.geoapify.com/v2/places?categories=airport&apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&limit=20&filter=circle%3A151.2082848%2C-33.8698439%2C50000&bias=proximity%3A151.2082848%2C-33.8698439
https://api.geoapify.com/v2/places?categories=airport&apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&limit=20&filter=circle%3A144.9631608%2C-37.8142176%2C50000&bias=proximity%3A144.9631608%2C-37.8142176
https://api.geoapify.com/v2/places?categories=airport&apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&limit=20&filter=circle%3A115.8605801%2C-31.9558964%2C50000&bias=proximity%3A115.8605801%2C-31.9558964
https://api.geoapify.com/v2/places?categories=airport&apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&limit=20&filter=circle%3A145.7721854%2C-16.9206657%2C50000&bias=proximity%3A145.7721854%2C-16.9206657
https://api.geoapify.com/v2/places?categories=airport&apiKey=2ac05097e2d34a9eb6562ddfe2c61dd9&limit=20&filter=circle%3A138.5999312%2C-34.9281805%2C50000&bias=proximity%3A138.5999312%2C

In [7]:
# Save Data to CSV
